In [11]:
from pathlib import Path

md_text = Path("files/coffee_shops_sf.md").read_text(encoding="utf-8")
print(md_text[:500])  # preview first 500 chars

# San Francisco Coffee Shops Research

## Top Coffee Shops in San Francisco

### Blue Bottle Coffee
- **Founded**: 2002 in Oakland, expanded to SF
- **Specialty**: Single-origin coffee, pour-over methods
- **Notable**: Known for freshly roasted beans, minimalist aesthetic
- **Locations**: Multiple SF locations including Ferry Building

### Philz Coffee
- **Founded**: 2003 in San Francisco
- **Specialty**: Custom blended coffee, personalized service
- **Notable**: Founded by Phil Jaber, known for


In [12]:
import uuid
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


In [17]:

# Initialize Model
model = init_chat_model("google_genai:models/gemini-2.5-flash-lite")

# --- 1. TypedDicts (The Internal Memory) ---
class Checkpoint(TypedDict):
    id: str
    name: str
    objective: str  # Matches Pydantic field below

class State(TypedDict):
    report: str
    checkpoints: list[Checkpoint]

# --- 2. Pydantic Models (The LLM Interface) ---
class CheckpointItem(BaseModel):
    name: str = Field(description="The name of the checkpoint")
    objective: str = Field(description="the specific objective of the checkpoint") # FIXED: typo 'objetive' -> 'objective'

class CheckpointResponse(BaseModel):
    checkpoints: List[CheckpointItem]

# Setup LLM
model_with_structured_output = model.with_structured_output(CheckpointResponse)

In [18]:
# --- 3. The Node ---
def generate(state: State):
    report = state['report']
    
    # Fixed variable name spelling
    response = model_with_structured_output.invoke(f"extract and breakdown this report {report} into learning checkpoints")
    
    clean_checkpoints = []
    for item in response.checkpoints:
        data = item.model_dump()
        
        # Inject ID
        data['id'] = str(uuid.uuid4())
        
        clean_checkpoints.append(data)
    
    # FIXED: Key must match State definition exactly ("checkpoints")
    return {"checkpoints": clean_checkpoints} 

# --- 4. Build Graph ---
builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_edge(START, "generate")
builder.add_edge("generate", END)

graph = builder.compile()

# --- 5. Run ---
# Define the variable first
# report_var = "This is a report about learning Python." 
result = graph.invoke({"report": md_text})

print(result)

{'report': '# San Francisco Coffee Shops Research\n\n## Top Coffee Shops in San Francisco\n\n### Blue Bottle Coffee\n- **Founded**: 2002 in Oakland, expanded to SF\n- **Specialty**: Single-origin coffee, pour-over methods\n- **Notable**: Known for freshly roasted beans, minimalist aesthetic\n- **Locations**: Multiple SF locations including Ferry Building\n\n### Philz Coffee\n- **Founded**: 2003 in San Francisco\n- **Specialty**: Custom blended coffee, personalized service\n- **Notable**: Founded by Phil Jaber, known for unique blends like "Tesora"\n- **Locations**: Original location in Mission District, now citywide\n\n### Sightglass Coffee\n- **Founded**: 2009\n- **Specialty**: Third-wave coffee, roasting transparency\n- **Notable**: Large roastery with viewing windows, industrial aesthetic\n- **Location**: SOMA district flagship location\n\n### Ritual Coffee Roasters\n- **Founded**: 2005\n- **Specialty**: Direct trade relationships, seasonal offerings\n- **Notable**: Pioneer in San F

In [19]:
result

{'report': '# San Francisco Coffee Shops Research\n\n## Top Coffee Shops in San Francisco\n\n### Blue Bottle Coffee\n- **Founded**: 2002 in Oakland, expanded to SF\n- **Specialty**: Single-origin coffee, pour-over methods\n- **Notable**: Known for freshly roasted beans, minimalist aesthetic\n- **Locations**: Multiple SF locations including Ferry Building\n\n### Philz Coffee\n- **Founded**: 2003 in San Francisco\n- **Specialty**: Custom blended coffee, personalized service\n- **Notable**: Founded by Phil Jaber, known for unique blends like "Tesora"\n- **Locations**: Original location in Mission District, now citywide\n\n### Sightglass Coffee\n- **Founded**: 2009\n- **Specialty**: Third-wave coffee, roasting transparency\n- **Notable**: Large roastery with viewing windows, industrial aesthetic\n- **Location**: SOMA district flagship location\n\n### Ritual Coffee Roasters\n- **Founded**: 2005\n- **Specialty**: Direct trade relationships, seasonal offerings\n- **Notable**: Pioneer in San F